In [ ]:
import re
import csv
import os
import json
import datetime
import typing as T
import functools
import pickle
from copy import deepcopy

import torch
import plotly.express as px
import pytorch_lightning as pl
import mlflow
import pandas as pd

import mtgradient
from mtgradient import processing, datasets, models
from mtgradient.card_initializer import featurize_cards, load_raw_card_data
from mtgradient.constants import PlayerRank


In [ ]:
# ! python -m pip install jupyter-black

# import jupyter_black

# jupyter_black.load(lab=False)

In [ ]:
# ! pip install mlflow
mlflow.pytorch.autolog(log_models=False, log_every_n_step=100)

In [ ]:
###################################
# Draft config
# -- location of raw data
# -- location to cache dataset
# -- time split
###################################

# draft_csv_path = "data/draft_data_public.NEO.PremierDraft.csv"
# draft_csv_path = "data/HBG/draft_data_public.HBG.PremierDraft.csv"
# draft_csv_path = "data/SNC/draft_data_public.SNC.PremierDraft.csv"
# draft_csv_path = "data/DMU/draft_data_public.DMU.PremierDraft.csv"
# draft_csv_path = "tests/testdata/snc_test.csv"
# draft_csv_path = "tests/testdata/test_premier_draft.csv"

# cache_path = "data/cached/neo_premier_draft"
# cache_path = "data/cached/hbg_premier_draft/"
# cache_path = "data/cached/snc_premier_draft"
# cache_path = "data/cached/dmu_premier_draft"
# cache_path = "tests/testdata/"

# test_split_cutoff = pd.Timestamp(datetime.date(2022, 3, 11))
# test_split_cutoff = pd.Timestamp(datetime.date(2022, 7, 28))

# val_split_cutoff = pd.Timestamp(datetime.date(2022, 9, 16))
# test_split_cutoff = pd.Timestamp(datetime.date(2022, 9, 17))
# recent_game_cutoff = val_split_cutoff - datetime.timedelta(3)

# data_params = {
#     "data_path": draft_csv_path,
#     "val_split_cutoff": val_split_cutoff,
#     "test_split_cutoff": test_split_cutoff,
#     "recent_game_cutoff": recent_game_cutoff,
# }

# LOAD_CACHED = True

In [ ]:
# if not LOAD_CACHED:
#     parsed_data, card_ids = processing.parse_csv(draft_csv_path, verbose=True)
#     os.makedirs(cache_path, exist_ok=True)
#     processing.persist_processed_dataset(cache_path, parsed_data, card_ids)
# else:
#     parsed_data, card_ids = processing.load_processed_dataset(cache_path)

In [ ]:
# draft_times = [[v["draft_time"]] for k, v in parsed_data.items()]
# px.histogram(pd.DataFrame(draft_times, columns=["t"]), x="t")

In [ ]:
# train_subset = {
#     k: v
#     for k, v in parsed_data.items()
#     if v.get("draft_time", datetime.date(1999, 9, 9)) < test_split_cutoff
# }
# val_subset = {
#     k: v
#     for k, v in parsed_data.items()
#     if v.get("draft_time", datetime.date(1999, 9, 9)) >= val_split_cutoff
#     and v.get("draft_time", datetime.date(1999, 9, 9)) < test_split_cutoff
# }
# test_subset = {
#     k: v
#     for k, v in parsed_data.items()
#     if v.get("draft_time", datetime.date(1999, 9, 9)) >= test_split_cutoff
# }

# train_draft_dataset = datasets.DraftDataset(train_subset, recent_game_cutoff)
# val_draft_dataset = datasets.DraftDataset(val_subset, val_split_cutoff, use_all=True)
# test_draft_dataset = datasets.DraftDataset(test_subset, test_split_cutoff, use_all=True)


In [ ]:
# concat = torch.utils.data.ConcatDataset([train_draft_dataset, train_draft_dataset2])
# itr = iter(concat)

# tdl = torch.utils.data.DataLoader(
#     concat,
#     batch_size=20,
#     shuffle=True,
#     drop_last=True,
#     collate_fn=models.collate_batch,
#     num_workers=2,
#     pin_memory=False,
# )
# next(iter(tdl))

In [ ]:
# dataset_config = [
#     {
#         "set_id": "DMU",
#         "datasets": [
#             {
#                 "format": "premier",
#                 "format_id_int": 0,
#                 "file": "data/DMU/draft_data_public.DMU.PremierDraft.csv",
#             },
#             {
#                 "format": "traditional",
#                 "format_id_int": 1,
#                 "file": "data/DMU/draft_data_public.DMU.TradDraft.csv",
#                 "training_only": True,
#             },
#         ],
#         "set_id_int": 0,
#     },
#     {
#         "set_id": "SNC",
#         "datasets": [
#             {
#                 "format": "premier",
#                 "format_id_int": 0,
#                 "file": "data/SNC/draft_data_public.SNC.PremierDraft.csv",
#                 "training_only": True,
#             },
#         ],
#         "set_id_int": 1,
#     },
# ]

# dataset_config = [
#     {
#         "set_id": "MID",
#         "datasets": [
#             {
#                 "format": "premier",
#                 "format_id_int": 0,
#                 "file": "data/MID/draft_data_public.MID.PremierDraft.csv",
#             },
#         ],
#         "set_id_int": 0,
#     },
# ]


# dataset_config = [
#     {
#         "set_id": "VOW",
#         "datasets": [
#             {
#                 "format": "premier",
#                 "format_id_int": 0,
#                 "file": "data/VOW/draft_data_public.VOW.PremierDraft.csv",
#             },
#             {
#                 "format": "quick",
#                 "format_id_int": 1,
#                 "file": "data/VOW/draft_data_public.VOW.QuickDraft.csv",
#                 "training_only": True,
#             },
#         ],
#         "set_id_int": 0,
#     },
#     #         {
#     #         "set_id": "SNC",
#     #         "datasets": [
#     #             {
#     #                 "format": "premier",
#     #                 "format_id_int": 0,
#     #                 "file": "data/SNC/draft_data_public.SNC.PremierDraft.csv",
#     #                 "training_only": True,
#     #             },
#     #         ],
#     #         "set_id_int": 1,
#     #     },
# ]


# dataset_config = [
#     {
#         "set_id": "NEO",
#         "datasets": [
#             {
#                 "format": "premier",
#                 "format_id_int": 0,
#                 "file": "data/NEO/draft_data_public.NEO.PremierDraft.csv",
#             }
#         ],
#         "set_id_int": 0,
#     },
    #         {
    #         "set_id": "SNC",
    #         "datasets": [
    #             {
    #                 "format": "premier",
    #                 "format_id_int": 0,
    #                 "file": "data/SNC/draft_data_public.SNC.PremierDraft.csv",
    #                 "training_only": True,
    #             },
    #         ],
    #         "set_id_int": 1,
    #     },
# ]


dataset_config = [
    {
        "set_id": "ONE",
        "datasets": [
            {
                "format": "premier",
                "format_id_int": 0,
                "file": "data/ONE/draft_data_public.ONE.PremierDraft.csv",
            }
        ],
        "set_id_int": 0,
    },
]


sets_params = {"set_names": ",".join([i["set_id"] for i in dataset_config])}

In [ ]:
# date_df = pd.read_csv(
#     "data/NEO/draft_data_public.NEO.PremierDraft.csv", usecols=["draft_time"]
# )

# px.histogram(date_df, x="draft_time")

In [ ]:
check = datasets.MultiDataset(dataset_config)

train_draft_dataset = torch.utils.data.ConcatDataset(check.train_datasets)
val_draft_dataset = torch.utils.data.ConcatDataset(check.val_datasets)
test_draft_dataset = torch.utils.data.ConcatDataset(check.test_datasets)

In [ ]:
card_ids = check.card_ids

In [ ]:
len(train_draft_dataset), len(val_draft_dataset), len(test_draft_dataset)

In [ ]:
with open("data/cached/one_only.pkl", "wb") as f:
    pickle.dump(check, f)

In [ ]:
with open("data/cached/one_only.pkl", "rb") as f:
    check = pickle.load(f)
card_ids = check.card_ids
train_draft_dataset = torch.utils.data.ConcatDataset(check.train_datasets)
val_draft_dataset = torch.utils.data.ConcatDataset(check.val_datasets)
test_draft_dataset = torch.utils.data.ConcatDataset(check.test_datasets)

In [ ]:
# data_out = [draft["draft_time"] for draft in check.train_datasets[0].dataset.values()]
# df_out = pd.DataFrame(data_out, columns=["draft_time"])

In [ ]:
data_out = ([draft["draft_time"] for draft in check.train_datasets[0].dataset.values()] +
    [draft["draft_time"] for draft in check.val_datasets[0].dataset.values()] +
           [draft["draft_time"] for draft in check.test_datasets[0].dataset.values()]   
           )
df_out = pd.DataFrame(data_out, columns=["draft_time"])


In [ ]:
px.histogram(df_out, x="draft_time", cumulative=True, histnorm="percent")

In [ ]:
# load the model and ensure its forward method works
N_CARDS = 1000
EMB_DIM = 512

params = {
    "n_cards": N_CARDS,
    "emb_dim": EMB_DIM,
    "n_cards_in_pack": 16,
    "n_steps": 30000,
}
model = models.DraftTransformer(
    **params,
).to("cuda:0")


card_data_df = load_raw_card_data(
    "data/oracle-cards-20230619090235.json", "data/", refresh=False
)
feature_vectors = featurize_cards(card_data_df, card_ids, N_CARDS, EMB_DIM)
model.init_embedding(feature_vectors)

print(
    model(
        models.collate_batch([next(iter(train_draft_dataset))], device="cuda:0"),
    )
)

In [ ]:
mlflow.pytorch.autolog(log_models=False, log_every_n_step=100)
try:
    mlflow.end_run()
except Exception as e:
    pass
mlflow.start_run()
mlflow.log_params(sets_params)
# mlflow.log_params({k: str(v) for k, v in params.items()})
# mlflow.log_params({k: str(v) for k, v in data_params.items()})
# mlflow.log_metrics(
#     {"len_val_subset": len(val_subset), "len_train_subset": len(train_subset)}
# )

# swa_args = dict(swa_lrs=5e-6, annealing_epochs=10, swa_epoch_start=25)
# mlflow.log_params({k: str(v) for k, v in swa_args.items()})

# swa = pl.callbacks.StochasticWeightAveraging(**swa_args)

checkpoint = pl.callbacks.ModelCheckpoint(save_weights_only=True, filename="model")
trainer = pl.Trainer(
    accelerator='gpu',
    max_steps=model.n_steps,
    #     profiler="simple",
    precision=16,
    #     callbacks=[checkpoint, swa],
    callbacks=[checkpoint],
    #     benchmark=True,
    check_val_every_n_epoch=1,
    #     limit_val_batches=1,
    #     overfit_batches=1,
)


train_dl = torch.utils.data.DataLoader(
    train_draft_dataset,
    batch_size=200,
    shuffle=True,
    drop_last=True,
    collate_fn=models.collate_batch,
    num_workers=2,
    pin_memory=False,
)
val_dl = torch.utils.data.DataLoader(
    val_draft_dataset,
    batch_size=200,
    shuffle=False,
    drop_last=False,
    collate_fn=models.collate_batch,
    num_workers=2,
    pin_memory=False,
)


trainer.fit(model, train_dl, val_dl)

In [ ]:
dp = checkpoint.dirpath
fn = os.listdir(trainer.checkpoint_callback.dirpath)[0]
checkpoint_path = os.path.join(dp, fn)

# log our model weights
mlflow.log_artifact(checkpoint_path)

# log the card ids
card_id_path = f"{dp}/card_ids.json"
with open(card_id_path, "w") as f:
    json.dump(card_ids, f)

mlflow.log_artifact(card_id_path)

# log the dataset config
ds_config_path = f"{dp}/dataset_config.json"
with open(ds_config_path, "w") as f:
    json.dump(dataset_config, f)
mlflow.log_artifact(ds_config_path)

# log the model params/config
model_config_path = f"{dp}/model_config.json"
with open(model_config_path, "w") as f:
    json.dump(params, f)

mlflow.log_artifact(model_config_path)

In [ ]:
run = mlflow.active_run()
run.info.run_id

In [ ]:
# ! rm -rf artifacts/SNC/
# ! cp -r mlruns/0/6f8fd2c4b705471791a1010c8b06d2e7/artifacts/ artifacts/SNC/

In [ ]:
# with open("artifacts/SNC/model_config.json", "r") as f:
#     m_config = json.load(f)
# loaded = models.DraftTransformer.load_from_checkpoint(
#     "artifacts/SNC/model-v1.ckpt", **m_config
# )
# t = pl.Trainer(
#     gpus=[0],
# )


test_dl = torch.utils.data.DataLoader(
    test_draft_dataset,
    batch_size=200,
    shuffle=False,
    drop_last=False,
    collate_fn=models.collate_batch,
    num_workers=2,
    pin_memory=False,
)


trainer.test(model, test_dataloaders=[test_dl])

In [ ]:

by_rank = pd.DataFrame(model.test_data, columns=["rank_ind", "pick_loss", "maindeck_rate_loss"])
by_rank["rank_ind"] = by_rank["rank_ind"].apply(int)
by_rank["rank"] = by_rank["rank_ind"].apply(lambda x: PlayerRank(x).name)


In [ ]:
by_rank.groupby(["rank_ind", "rank"]).mean()

In [ ]:
px.box(
    by_rank.sort_values("rank_ind"),
    x="rank",
    y="pick_loss",
    range_y=(0, 1.25),
    title="Cross entropy loss for card pick by rank",
)

In [ ]:
px.box(
    by_rank.sort_values("rank_ind"),
    x="rank",
    y="maindeck_rate_loss",
    range_y=(0, 1.0),
    title="Maindeck loss for card pick by rank",
)

In [ ]:
test_data_df = pd.DataFrame(
    model.test_data, columns=["rank_id", "pick_loss", "maindeck_loss"]
)
test_data_df

In [ ]:
dts = set()
for draft_id, draft in check.train_datasets[0].dataset.items():
    if draft.get("rank") == "0":
        break

In [ ]:
for thing, draft in check.train_datasets[0].dataset.items():
    print(
        [
            i
            for n, i in enumerate(draft["pick_data"])
            if draft["maindeck_rates"][n] > 0.5
        ]
    )
    break

In [ ]:
from scipy.sparse import csr_matrix
from sklearn.decomposition import MiniBatchSparsePCA, SparsePCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.pipeline import Pipeline
import numpy as np


class ArchetypeBuilder:
    def __init__(
        self, n_archetypes: int = 15, n_components: int = 50, alpha: float = 0.5
    ):
        self.n_archetypes = n_archetypes
        self.pca = MiniBatchSparsePCA(
            n_components=n_components, alpha=alpha, batch_size=100
        )
        #         self.pca = PCA(n_components=n_components)
        #         self.pca = SparsePCA(n_components=n_components)
        self.clusterer = MiniBatchKMeans(n_clusters=n_archetypes)
        self.pipeline = Pipeline([("pca", self.pca), ("clustering", self.clusterer)])

    #         self.pipeline = Pipeline([("clustering", self.clusterer)])

    def create_sparse_played_deck_dataset(self, dataset: datasets.DraftDataset):
        row_ids: T.List[int] = []
        col_ids: T.List[int] = []
        draft_ids: T.List[str] = []

        for draft_id, draft in dataset.items():
            draft_ids.append(draft_id)
            draft_ind = len(draft_ids) - 1
            for n, i in enumerate(draft["pick_data"]):
                if draft["maindeck_rates"][n] > 0.5:
                    row_ids.append(draft_ind)
                    col_ids.append(i)
        result = csr_matrix((np.ones(len(col_ids)), (row_ids, col_ids)))
        return result

    def fit(self, dataset: datasets.DraftDataset):
        data = self.create_sparse_played_deck_dataset(dataset)
        result = self.pipeline.fit(data.todense())
        return result

In [ ]:
archetype_builder = ArchetypeBuilder()

In [ ]:
result = archetype_builder.create_sparse_played_deck_dataset(
    check.train_datasets[0].dataset
)

In [ ]:
archetype_builder.fit(check.train_datasets[0].dataset)

In [ ]:
card_ids_inv = {v: k for k, v in check.card_ids.items()}
for archetype_feature in archetype_builder.pca.components_:
    print([card_ids_inv[n] for n, i in enumerate(archetype_feature) if i > 0.05])
    print("*" * 50)

In [ ]:
archetype_builder.clusterer.cluster_centers_

In [ ]:
card_ids_inv = {v: k for k, v in check.card_ids.items()}
for archetype_feature in archetype_builder.clusterer.cluster_centers_:
    print([card_ids_inv[n] for n, i in enumerate(archetype_feature) if i > 0.5])
    print("*" * 50)

In [ ]:
import numpy as np
from sklearn.manifold import TSNE

fv = feature_vectors[: np.max(list(card_ids.values()))]

In [ ]:
tsned_vecs = TSNE().fit_transform(fv)

In [ ]:
tdf = pd.DataFrame(tsned_vecs, columns=["x1", "x2"]).reset_index(drop=False)
inv_map = {v: k for k, v in card_ids.items()}
tdf["card"] = tdf["index"].apply(lambda x: inv_map.get(x, ""))

In [ ]:
tdf

In [ ]:
px.scatter(tdf, x="x1", y="x2", hover_data=["card"])